Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Prepare)

Today's guided module project will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills. The competition will begin

## Learning Objectives
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Overview

Sklearn pipelines allow you to stitch together multiple components of a machine learning process. The idea is that you can pass you raw data and get predictions out of the pipeline. This ability to pass raw input and receive a prediction from a singular class makes pipelines well suited for production, because you can pickle a a pipeline without worry about other data preprocessing steps. 

*Note:* Each time we call the pipeline during grid search, each component is fit again. The vectorizer (tf-idf) is transforming our entire vocabulary during each cross-validation fold. That transformation adds significant run time to our grid search. There *might* be interactions between the vectorizer and our classifier, so we estimate their performance together in the code below. However, if your goal is to reduce run time. Train your vectorizer separately (ie out of the grid-searched pipeline). 

In [1]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# Dataset
from sklearn.datasets import fetch_20newsgroups

# 2 categories to class today
categories = ['alt.atheism',
              'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)

In [15]:
# Y data
data.target[:5]

array([0, 0, 0, 1, 1])

In [16]:
data.target_names[:5]

['alt.atheism', 'talk.religion.misc']

In [8]:
len(data.data)

857

In [4]:
#print out data sample
# X data/ input
data.data[:3]

['From: mangoe@cs.umd.edu (Charley Wingate)\nSubject: Benediktine Metaphysics\nLines: 24\n\nBenedikt Rosenau writes, with great authority:\n\n>     IF IT IS CONTRADICTORY IT CANNOT EXIST.\n\n"Contradictory" is a property of language.  If I correct this to\n\n\n      THINGS DEFINED BY CONTRADICTORY LANGUAGE DO NOT EXIST\n\nI will object to definitions as reality.  If you then amend it to\n\n      THINGS DESCRIBED BY CONTRADICTORY LANGUAGE DO NOT EXIST\n\nthen we\'ve come to something which is plainly false.  Failures in\ndescription are merely failures in description.\n\n(I\'m not an objectivist, remember.)\n\n\n-- \nC. Wingate        + "The peace of God, it is no peace,\n                  +    but strife closed in the sod.\nmangoe@cs.umd.edu +  Yet, brothers, pray for but one thing:\ntove!mangoe       +    the marv\'lous peace of God."\n',
 'Subject: Re: There must be a creator! (Maybe)\nFrom: halat@pooh.bears (Jim Halat)\nReply-To: halat@pooh.bears (Jim Halat)\nLines: 24\n\nIn article

In [9]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

In [20]:
# Define the Pipeline
pipe = Pipeline([
                  #Vectorizer (i.e. data transformation)
                 ('vect', vect),
                 # Classifier (Pipeline is expecting that the pip ends with a model)
                 ('clf', rfc)
                ])


In [19]:
# h-parameter dict for BOTH our vectorizer and our ML model 
# here we will determine which tfidf parameter values lead to the best performing model
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': ( 2, 10),
    'vect__max_features': (500, 1000),
    'clf__n_estimators':(10, 100),
    'clf__max_depth':(15, 20)
}

grid_search = GridSearchCV(pipe, 
                           parameters, 
                           cv=5, 
                           n_jobs=-1, 
                           verbose=1)

X = data.data
Y = data.target

grid_search.fit(X, Y)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:   19.9s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [12]:
grid_search.best_score_

0.9136610907112743

In [18]:
grid_search.predict(['I love god. ', 'I only believe in science and I am not religious.'])

array([1, 1])

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

## Challenge

You're trying to achieve 75% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Overview

![](https://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1538411402/image3_maagmh.png)

In [29]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)

In [36]:
# a nice default starter set for param values 
params = { 
    'lsi__svd__n_components': [10, 100, 250],
    'lsi__vect__max_df':[.9,  1.0],
    'clf__n_estimators':[10, 100, 250]
}

In [37]:
# LSI
lsi = Pipeline([('vect', vect), # vectorize data, returns a doc-term matrix (dtm)
                ('svd', svd)]) # pass dtm into svd for dim reduction and feat eng (i.e. discover hidden topics in dataset)


# Pipe
pipe = Pipeline([('lsi', lsi), # data transformations
                 ('clf', rfc)]) # model fitting

In [38]:
print(pipe)

Pipeline(memory=None,
         steps=[('lsi',
                 Pipeline(memory=None,
                          steps=[('vect',
                                  TfidfVectorizer(analyzer='word', binary=False,
                                                  decode_error='strict',
                                                  dtype=<class 'numpy.float64'>,
                                                  encoding='utf-8',
                                                  input='content',
                                                  lowercase=True, max_df=1.0,
                                                  max_features=None, min_df=1,
                                                  ngram_range=(1, 2), norm='l2',
                                                  preprocessor=None,
                                                  smooth_idf=True,
                                                  stop_words='english',
                                                  strip_a

In [39]:
# Fit
grid_search = GridSearchCV(pipe, # pass in pipe object
                           params, # h-param dict
                           cv=2, # reduce this to 2 or 3 in shorten run time 
                           n_jobs=-1, # make sure this is set to -1 to shorten run time 
                           verbose=1)

X = data.data
Y = data.target
grid_search.fit(X, Y)

Fitting 2 folds for each of 18 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  1.7min finished


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

In [42]:
grid_search.best_params_

{'clf__n_estimators': 250,
 'lsi__svd__n_components': 100,
 'lsi__vect__max_df': 0.9}

In [43]:
grid_search.best_score_

0.8996334662222513

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

# Overview

In [44]:
import spacy

# load in pre-trained w2v model 
nlp = spacy.load("en_core_web_lg")

In [47]:
doc = nlp("Two bananas in pyjamas")

In [49]:
bananas_vector = doc.vector
print(len(bananas_vector))

300


In [51]:
def get_word_vectors(docs):
    """
    This serves as both our tokenizer and vectorizer. 
    Returns a list of word vectors, i.e. our doc-term matrix
    """
    return [nlp(doc).vector for doc in docs]

In [52]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('./Kaggle Data/train.csv')
test = pd.read_csv('./Kaggle Data/test.csv')

In [72]:
test.head()

,id,description,ratingCategory
0,3461,\nStyle: Speyside single malt scotch Color: Wa...,1
1,2604,"\nVery bright and lively, with a nice balance ...",1
2,3341,\nA new oloroso-forward Chivas positioned to s...,1
3,3764,\nAged in bourbon casks and then enhanced in R...,1
4,2306,\nThere is a freshness to the wood on the nose...,1


In [53]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [55]:
train.shape

(4087, 3)

In [57]:
# take a sub-sample in decrease run time
n_samples = 1000
train = train.sample(n=n_samples)

In [58]:
# create our doc-term matrices 

# raw text data for train and test sets
X_train_text = train['description']
X_test_text = test['description']

# transform raw data into doc-term matrices for train and test sets 
X_test = get_word_vectors(X_test_text)
X_train = get_word_vectors(X_train_text)

# save ratings to y vector
y_train = train['ratingCategory']

In [62]:
rfc = RandomForestClassifier(oob_score=True)

In [63]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [64]:
# training set accuracy is 100% 
rfc.score(X_train, y_train)

1.0

In [66]:
# this is the out-of-bag score which we can treat as hold out score (i.e a test score) 
rfc.oob_score_

0.709

In [69]:
test['ratingCategory'] = rfc.predict(X_test)

In [70]:
test[['id', 'ratingCategory']].to_csv('testSolutionSubmission.csv', header=True, index=False)

In [71]:
rfc.score(X_test, test['ratingCategory'])

1.0

## Follow Along

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Review

To review this module: 
* Continue working on the Kaggle competition
* Find another text classification task to work on